In [1]:
using Pkg; Pkg.activate(".")
using HypergraphModularity

using Random
using StatsBase

 Activating environment at `~/codes/hypergraph_modularities_code/Project.toml`
┌ Info: Precompiling HypergraphModularity [0c934d27-dd44-49d7-950f-bd4be7819e54]
└ @ Base loading.jl:1260
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

┌ Warning: Replacing docs for `HypergraphModularity.ConstructAdj :: Tuple{SparseArrays.SparseMatrixCSC,Int64}` in module `HypergraphModularity`
└ @ Base.Docs docs/Docs.jl:229
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Replacing docs for `HypergraphModularity.LamCCobj :: Tuple{SparseArrays.SparseMatrixCSC,Any,Any,Any}` in module `HypergraphModularity`
└ @ Base.Docs docs/Docs.jl:229
  ** incremental compilation may be fatally broken for this module **

┌ Warning: Replacing docs for `HypergraphModularity.renumber :: Tuple{Array{Int64,1},Array{Array{Int64,1},1}

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** i

In [2]:
identity(p::Vector{Int64}) = p

function sum_of_ext_degs(p::Vector{Int64})
    soed = length(p) - 1
    return (sum(p), soed)
end

function all_or_nothing(p::Vector{Int64})
    is_aon = length(p) == 1
    return (sum(p), is_aon)
end

function rainbow(p::Vector{Int64})
    is_rainbow = length(p) == sum(p) && length(p) > 1
    return (sum(p), is_rainbow)
end


function MLE_ll(Hyp, Z, agg)
    Ω̂ = estimateΩEmpirically(Hyp, Z; min_val=0, aggregator=agg)
    return Float64(sum(L(Hyp, Z, Ω̂; α=0, bigInt=true)))    
end

function dyadic_MLE_ll(Hyp, Z, weighted::Bool)
    ω_in, ω_out = 
        computeDyadicResolutionParameter(Hyp, Z; mode="ω", weighted=weighted)
    return Float64(dyadicLogLikelihood(Hyp, Z, ω_in, ω_out, weighted))
end

dyadic_MLE_ll (generic function with 1 method)

In [3]:
function sub_hypergraph(h::hypergraph, labels, in_subset::Vector{Bool})
    # Get new set of edges
    new_edges = []
    for (sz, edges) in h.E
        for (edge, val) in edges
            new_edge = filter(v -> in_subset[v], edge)
            if length(new_edge) > 1
               push!(new_edges, new_edge)
            end
        end
    end
    
    # renumbering
    node_map = Dict{Int64,Int64}()
    for (i, val) in enumerate(in_subset)
        if val
            node_map[i] = length(node_map) + 1
        end
    end
    
    renumber_edge(e) = [node_map[v] for v in e]
    renumbered_new_edges = [renumber_edge(e) for e in new_edges]
    
    subE = Dict{Integer, Dict}()
    for edge in renumbered_new_edges
        sz = length(edge)
        if !haskey(subE, sz)
            subE[sz] = Dict{}()
        end
        subE[sz][edge] = 1
    end
    
    n = length(node_map)
    subD = zeros(Int64, n)
    for (sz, edges) in subE
        for (e, _) in edges
            subD[e] .+= 1
        end
    end
    
    return hypergraph(1:n, subE, subD), labels[in_subset], node_map
end

sub_hypergraph (generic function with 1 method)

In [4]:
# merged labels
function merge_gender_labels(names, labels)
    merge_names = String[]
    merge_labels = Int64[]
    split_names = [split(name) for name in names]
    for label in labels
        merge_name = split_names[label][1]
        if !(merge_name in merge_names)
           push!(merge_names, merge_name) 
        end
        ind = findall(merge_names .== merge_name)[1]
        push!(merge_labels, ind)
    end
    return merge_names, merge_labels
end

merge_gender_labels (generic function with 1 method)

In [5]:
for dataset in ["primary-school", "high-school"]
    H, labels = read_hypergraph_data("contact-$dataset-classes-gender", 10)
    names = read_hypergraph_label_names("contact-$dataset-classes-gender")

    # Throw out data where gender is unknown
    keep = ones(Bool, length(labels))
    for (i, label) in enumerate(labels)
        keep[i] = !occursin("Unknown", names[label])
    end

    merge_names, merge_labels = merge_gender_labels(names, labels)

    subH, sublabels1, _ = sub_hypergraph(H, labels, keep)
    _, sublabels2, _ = sub_hypergraph(H, merge_labels, keep)
    
    println("dataset = ", dataset)
    
    println("Polyadic MLE LL (with genders): ", 
            MLE_ll(subH, sublabels1, all_or_nothing))
    println("Polyadic MLE LL (just classes): ", 
            MLE_ll(subH, sublabels2, all_or_nothing))
    println("")

    println("Weighted Dyadic MLE LL (with genders): ",     
            dyadic_MLE_ll(subH, sublabels1, true))
    println("Weighted Dyadic MLE LL (just classes): ",     
            dyadic_MLE_ll(subH, sublabels2, true))
    println("")

    println("Unweighted Dyadic MLE LL (with genders): ",     
            dyadic_MLE_ll(subH, sublabels1, false))
    println("Unweighted Dyadic MLE LL (just classes): ",     
            dyadic_MLE_ll(subH, sublabels2, false))    
    println("")
end

dataset = primary-school
Polyadic MLE LL (with genders): -470152.1789186721
Polyadic MLE LL (just classes): -54676.65251816025

Weighted Dyadic MLE LL (with genders): -9997.963342277139
Weighted Dyadic MLE LL (just classes): -6867.170603483688

Unweighted Dyadic MLE LL (with genders): -13550.94800509466
Unweighted Dyadic MLE LL (just classes): -7860.319734901284

dataset = high-school
Polyadic MLE LL (with genders): -92326.53363478063
Polyadic MLE LL (just classes): -27550.859421192486

Weighted Dyadic MLE LL (with genders): -3451.3748596152136
Weighted Dyadic MLE LL (just classes): 673.2689102861495

Unweighted Dyadic MLE LL (with genders): -4442.675727563024
Unweighted Dyadic MLE LL (just classes): 1740.0722419018912

